In [5]:
import random
import numpy as np
import copy
import mnist
import matplotlib.pyplot as plt
import time
import copy
import pickle

In [2]:
class Kmeans ():
    
    def __init__(self,org_data,k,T):
        self.org_data = org_data
        self.dels = []
        self.k = k
        self.T = T
        self.data = None
        self.centroids = None
        self.clusters = None
        self.loss = None
    
    
    def fit(self,data):
        self.data = data
        
        self.initialize()
        self.lloyds()
        #return np.array(fin_centroids),fin_clusters,loss
    
    
    def initialize(self):
    
        n = self.data.shape[0]
        d = self.data.shape[1]
        
        centroid = random.choice(self.data)
        self.centroids = np.array([centroid])

        for i in range(self.k-1):
            Probs = self.get_probs()
            idx = np.random.choice(n,p=Probs)
            centroid = self.data[idx]
            self.centroids = np.vstack([self.centroids,centroid])

        #return init_centroids
    
    def get_probs(self):
    
        Probs = np.zeros([len(self.data)])
        for i in range(len(self.data)):
            dis = np.linalg.norm(self.data[i] - self.centroids,axis=1)
            Probs[i] = np.min(dis)
        Probs = [dist**2 for dist in Probs]
        Probs = Probs/sum(Probs)
        return Probs
    
    
    def lloyds(self):
        T = self.T
        
        for it in range(self.T):
            self.find_clusters()
            self.find_centroids()
            
        #return centroids,clusters,loss
    
    
    def find_clusters(self):
        
        self.loss = 0
        self.clusters = {j:[] for j in range(self.k) }
        for i in range(len(self.data)):
            dis = np.linalg.norm(self.data[i] - self.centroids,axis=1)
            self.loss = self.loss + (np.min(dis))**2
            j = np.argmin(dis)
            self.clusters[j].append(i)
        
        #return clusters,loss
    
    
    def find_centroids(self):
        
        self.centroids = []
        for j in self.clusters.keys():
            if(len(self.clusters[j]) != 0):
                runn_sum = np.zeros(len(self.data[0]))
                count = 0
                for itm in self.clusters[j]: 
                    runn_sum  = runn_sum + self.data[itm]
                    count +=1
                centroid = runn_sum/count
                if(len(self.centroids) == 0):
                    self.centroids = np.vstack([centroid])
                else:
                    self.centroids = np.vstack([self.centroids,centroid])
            else:
                centroid = self.reassign_cluster()
                #print("type",type(self.centroids))
                if(len(self.centroids) == 0):
                    self.centroids = np.vstack([centroid])
                else:
                    self.centroids = np.vstack([self.centroids,centroid])

        self.centroids = np.array(self.centroids)
    
    def reassign_cluster(self):
        
        #print("reassign_cluster",centroids.shape)
        
        self.centroids = np.array(self.centroids)
        
        if(len(self.centroids) == 0):
            centroid = random.choice(self.data)
        else:
            Probs = self.get_probs()
            idx = np.random.choice(len(self.data),p=Probs)
            centroid = self.data[idx]

        return centroid
    
    
    def delete(self,idx):
        self.dels.append(idx)
        new_data = np.delete(self.org_data,self.dels,axis=0)
        self.fit(new_data)
    
    

In [33]:
class metadata:
    
    def __init__(self,centroids,q_centroids,phase,cluster_sizes):
        self.centroids = centroids
        self.q_centroids = q_centroids
        self.phase = phase
        self.cluster_sizes = cluster_sizes
        
class Q_Kmeans(Kmeans):
    
    def __init__(self,org_data,k,T,gamma,epsilon):
        Kmeans.__init__(self,org_data,k,T)
        self.gamma = gamma
        self.epsilon = epsilon
        self.metadata = None
        self.initial_centroids = None
        
    
    def qfit(self,data):
        self.data = data
        
        n = len(self.data)
        d = len(self.data[0])
        
        self.metadata = []
        
        self.initialize()
        self.initial_centroids = copy.deepcopy(self.centroids)
        
        self.find_clusters()
        cluster_sizes = [len(self.clusters[j]) for j in self.clusters.keys()]
        m0 = metadata(self.initial_centroids,self.initial_centroids,0,cluster_sizes)
        self.metadata.append(m0)
        
        for tau in range(self.T):
            prev_centroids = copy.deepcopy(self.centroids)
            self.find_centroids()
            
            self.imbalance_correction(prev_centroids)
            
            theta = np.random.random([d])
            centroids_hat = self.quantize(self.centroids,theta)
            
            # stored actual self values here
            actual_centroids = copy.deepcopy(self.centroids)
            actual_clusters = copy.deepcopy(self.clusters)
            actual_loss = self.loss
            
            # set centroids_hat as self.centroids
            self.centroids = centroids_hat
            self.find_clusters()
            
            cluster_sizes = [len(self.clusters[j]) for j in self.clusters.keys()]
            m = metadata(actual_centroids,self.centroids,theta,cluster_sizes)
            self.metadata.append(m)
            
            if(self.loss>=actual_loss):
                self.centroids = actual_centroids
                self.clusters = actual_clusters
                self.loss = actual_loss
                
                break
                
        
    def imbalance_correction(self,prev_centroids):
        n = len(self.data)
        
        for kappa in range(1,self.k+1):
            val1 = len(self.clusters[kappa-1])
            val2 = self.gamma*n/self.k
            if(val1 < val2):
                self.centroids[kappa-1] = (val1/val2)*self.centroids[kappa-1] + ((val2 - val1)/val2)*prev_centroids[kappa-1] 
    
    def quantize(self,c,theta):
        epsilon = self.epsilon
        c = c * 1/epsilon
        c = c + theta - 0.5
        c = np.round(c)
        c = c - theta + 0.5
        c = c * epsilon
        return c
    
    
    def qdelete(self,idx):
        self.dels.append(idx)
        n = len(self.data)
        
        p = self.org_data[idx]
        
        if p in self.initial_centroids:
            new_data = np.delete(self.org_data,self.dels,axis=0)
            self.qfit(new_data)
            #print("lolo")
            
        else:
            
            flag = 1
            for tau in range(len(self.metadata)-1):
    
                state = self.metadata[tau+1]
                c = state.centroids
                theta = state.phase
                cluster_sizes = state.cluster_sizes
                
                #p belongs to which cluster
                kappa = self.which_cluster(p,c)
                ck = c[kappa]
                
                #c_prevk
                prev_state = self.metadata[tau]
                #print("prev_state.q_centroids ",prev_state.q_centroids)
                c_prev = prev_state.q_centroids
                c_prevk = c_prev[kappa]
                
                #clustersize
                val1 = cluster_sizes[kappa] 
                
                # gamma imbalance check
                val2 = self.gamma*n/self.k
                if(val1 < val2):
                    lag = (val2 - val1)/val2
                    ck = (val1/val2)*ck
                    ck = ck + lag*c_prevk
                 
                
                #perturbing center by removing point p
                perturbed_ck = ck - p/val2
                
                #quantize
                cq = self.quantize(ck,theta)
                perturbed_cq = self.quantize(perturbed_ck,theta)
                c_prev[kappa] = perturbed_cq
                

                if not all(cq == perturbed_cq):
                    # re train ,return
                    print("retrain")
                    flag = 0
                    new_data = np.delete(self.org_data,self.dels,axis=0)
                    self.qfit(new_data)
                    break
                else:
                    state.cluster_sizes[kappa] = state.cluster_sizes[kappa]-1
                    new_state = metadata(c,c_prev,theta,state.cluster_sizes)
                    self.metadata[tau+1] = new_state
                    self.data = np.delete(self.org_data,self.dels,axis=0)
              
            if (flag):
                print("no retrain")
                
            print(idx,len(self.data))
            print()
            print()
                    
    def which_cluster(self,p,centroids):
        dis = np.linalg.norm(p - centroids,axis=1)
        kappa = np.argmin(dis)
        return kappa
        

In [4]:

class node:
    '''
    Class for a node of the tree used in the DC tree
    '''
    
    def __init__(self,lvl,parent=None):
        self.children = None
        self.lvl = lvl
        self.dataset = []
        self.centroids = []
        self.parent = parent
        
            
class tree:
    '''
    Class for the DC tree
    '''
    
    def __init__(self,w,h):
        '''
        Sets the tree parameters
        w: # of children for each non leaf node
        h: height of the DC tree
        '''
        self.w = w
        self.h = h
        self.levels = {i:[] for i in range(h)}
        self.root = None
        self.leaves = []
        self.which_leaf = None
        
        
    def initialize(self):
        '''
        Constructs the tree with tree parameters
        '''
        self.root = node(0)
        self.make_tree(self.root,self.w,self.h)
        self.levels[0]=[self.root]
        
        
    def make_tree(self,root,w,h):
        if(h>1):
            root.children = [ node(root.lvl+1,root) for i in range(w)]
            self.levels[root.lvl+1] = self.levels[root.lvl+1] + root.children
            if(h>2):
                for i in range(w):
                    self.make_tree(root.children[i],w,h-1)
            elif(h==2):
                self.leaves = self.leaves+root.children
            
        
    def lvl_traverse(self):
        for i in range(len(self.levels)):
            print(i,len(self.levels[i]))
            

##################################################################################


class DC_Kmeans():
    
    def __init__(self,org_data,k,T,w,h):
        self.org_data = org_data
        self.dels = []
        self.k = k
        self.T = T
        self.data = None
        self.centroids = None
        self.clusters = None
        self.loss = None
        self.w = w
        self.h = h
        self.tree = None
        
        
    def dcfit(self,data):
        self.data = data
        
        n = len(self.data)
        d = len(self.data[0])
        
        self.tree = tree(self.w,self.h+1)
        self.tree.initialize()
        
        self.tree.which_leaf = {i:0 for i in range(n)}
        
        for i in range(n):
            leaf = random.choice(self.tree.leaves)
            leaf.dataset.append(np.array(self.data[i]))
            self.tree.which_leaf[i]=leaf
            
        for l in range(self.h,-1,-1):
            for node in self.tree.levels[l]:
                km = Kmeans(np.array(node.dataset),self.k,self.T)
                km.fit(np.array(node.dataset))
                c,cl,loss = km.centroids,km.clusters,km.loss
                node.centroids = c
                if l>0:
                    node.parent.dataset.extend(c)
                else:
                    ## l = 0
                    kmf = Kmeans(self.data,self.k,self.T)
                    kmf.centroids,kmf.data = c,self.data
                    kmf.find_clusters()
                    
                    self.centroids = c
                    self.loss = kmf.loss
                    self.clusters = kmf.clusters
                    
                    
    def dcdelete(self,idx):
        self.dels.append(idx)
        
        n = len(self.data)
        d = len(self.data[0])
        
        self.data = np.delete(self.org_data,self.dels,axis=0)
        
        p = self.org_data[idx]
        
        node = self.tree.which_leaf[idx]
        ind = np.where(node.dataset == p)
        node.dataset = np.delete(node.dataset, ind[0][0],axis = 0)

        while (node.parent != None):
            for centroid in node.centroids :
                ind = np.where(node.parent.dataset == centroid)
                node.parent.dataset = np.delete(node.parent.dataset,ind[0][0],axis = 0)
            
            km = Kmeans(np.array(node.dataset),self.k,self.T)
            km.fit(np.array(node.dataset))
            node.centroids,cl,loss = km.centroids,km.clusters,km.loss
            node.parent.dataset = np.vstack([node.parent.dataset,node.centroids])
            node = node.parent
        
        km1 = Kmeans(np.array(node.dataset),self.k,self.T)
        km1.fit(np.array(node.dataset))
        node.centroids = km1.centroids
        
        km2 = Kmeans(self.data,self.k,self.T)
        km2.centroids,km2.data = node.centroids,self.data
        km2.find_clusters()
        
        self.centroids = km2.centroids
        self.clusters = km2.clusters
        self.loss = km2.loss



In [10]:
data_file = open("scaled_data.p",'rb')  
total_data = pickle.load(data_file)

In [14]:
print(total_data.keys())

dict_keys(['mnist', 'bot_attack', '4celltypes_10pca', 'postures', 'covtype_multiclass'])


In [25]:
mnist = total_data['mnist']
botattack = total_data['bot_attack']
celltype = total_data['4celltypes_10pca']
postures = total_data['postures']
covtype = total_data['covtype_multiclass']

In [21]:
print(bot_attack[0].shape)
print(bot_attack[1].shape)
print(bot_attack[2])

(1018298, 115)
(1018298,)
11


In [26]:
def make_data(datatuple):
    X = datatuple[0]
    y = datatuple[1]
    d = X.shape[1]
    k = datatuple[2]
    print(X.shape,k)
    return X,y,d,k

## celltype

In [38]:
X,y,d,k = make_data(celltype)

(12009, 10) 4


In [39]:
km = Kmeans(X,k,10)

st = time.time()
km.fit(X)
en = time.time()

centers,clusters,loss = km.centroids,km.clusters,km.loss

print(en-st)
print(loss)

2.249497890472412
188.40532190496404


In [40]:
qkm = Q_Kmeans(X,k,10,0.2,0.5)

st = time.time()
qkm.qfit(X)
en = time.time()

centers,clusters,loss = qkm.centroids,qkm.clusters,qkm.loss

print(en-st)
print(loss)

0.8435125350952148
279.7778874100176


In [41]:
dckm = DC_Kmeans(X,k,10,40,1)

st = time.time()
dckm.dcfit(X)
en = time.time()

centers,clusters,loss = dckm.centroids,dckm.clusters,dckm.loss

print(en-st)
print(loss)

2.7024662494659424
270.70732607670647


In [42]:
dels = random.sample(range(len(X)),1000)

In [35]:
st = time.time()
for d in range(len(dels)):
    print("processing deletion request {} ...".format(d+1))
    dckm.dcdelete(dels[d]) 
en = time.time()

print(en-st)

processing deletion request 1 ...
processing deletion request 2 ...
processing deletion request 3 ...
processing deletion request 4 ...
processing deletion request 5 ...
processing deletion request 6 ...
processing deletion request 7 ...
processing deletion request 8 ...
processing deletion request 9 ...
processing deletion request 10 ...
processing deletion request 11 ...
processing deletion request 12 ...
processing deletion request 13 ...
processing deletion request 14 ...
processing deletion request 15 ...
processing deletion request 16 ...
processing deletion request 17 ...
processing deletion request 18 ...
processing deletion request 19 ...
processing deletion request 20 ...
processing deletion request 21 ...
processing deletion request 22 ...
processing deletion request 23 ...
processing deletion request 24 ...
processing deletion request 25 ...
processing deletion request 26 ...
processing deletion request 27 ...
processing deletion request 28 ...
processing deletion request 2

processing deletion request 232 ...
processing deletion request 233 ...
processing deletion request 234 ...
processing deletion request 235 ...
processing deletion request 236 ...
processing deletion request 237 ...
processing deletion request 238 ...
processing deletion request 239 ...
processing deletion request 240 ...
processing deletion request 241 ...
processing deletion request 242 ...
processing deletion request 243 ...
processing deletion request 244 ...
processing deletion request 245 ...
processing deletion request 246 ...
processing deletion request 247 ...
processing deletion request 248 ...
processing deletion request 249 ...
processing deletion request 250 ...
processing deletion request 251 ...
processing deletion request 252 ...
processing deletion request 253 ...
processing deletion request 254 ...
processing deletion request 255 ...
processing deletion request 256 ...
processing deletion request 257 ...
processing deletion request 258 ...
processing deletion request 

processing deletion request 460 ...
processing deletion request 461 ...
processing deletion request 462 ...
processing deletion request 463 ...
processing deletion request 464 ...
processing deletion request 465 ...
processing deletion request 466 ...
processing deletion request 467 ...
processing deletion request 468 ...
processing deletion request 469 ...
processing deletion request 470 ...
processing deletion request 471 ...
processing deletion request 472 ...
processing deletion request 473 ...
processing deletion request 474 ...
processing deletion request 475 ...
processing deletion request 476 ...
processing deletion request 477 ...
processing deletion request 478 ...
processing deletion request 479 ...
processing deletion request 480 ...
processing deletion request 481 ...
processing deletion request 482 ...
processing deletion request 483 ...
processing deletion request 484 ...
processing deletion request 485 ...
processing deletion request 486 ...
processing deletion request 

processing deletion request 688 ...
processing deletion request 689 ...
processing deletion request 690 ...
processing deletion request 691 ...
processing deletion request 692 ...
processing deletion request 693 ...
processing deletion request 694 ...
processing deletion request 695 ...
processing deletion request 696 ...
processing deletion request 697 ...
processing deletion request 698 ...
processing deletion request 699 ...
processing deletion request 700 ...
processing deletion request 701 ...
processing deletion request 702 ...
processing deletion request 703 ...
processing deletion request 704 ...
processing deletion request 705 ...
processing deletion request 706 ...
processing deletion request 707 ...
processing deletion request 708 ...
processing deletion request 709 ...
processing deletion request 710 ...
processing deletion request 711 ...
processing deletion request 712 ...
processing deletion request 713 ...
processing deletion request 714 ...
processing deletion request 

processing deletion request 916 ...
processing deletion request 917 ...
processing deletion request 918 ...
processing deletion request 919 ...
processing deletion request 920 ...
processing deletion request 921 ...
processing deletion request 922 ...
processing deletion request 923 ...
processing deletion request 924 ...
processing deletion request 925 ...
processing deletion request 926 ...
processing deletion request 927 ...
processing deletion request 928 ...
processing deletion request 929 ...
processing deletion request 930 ...
processing deletion request 931 ...
processing deletion request 932 ...
processing deletion request 933 ...
processing deletion request 934 ...
processing deletion request 935 ...
processing deletion request 936 ...
processing deletion request 937 ...
processing deletion request 938 ...
processing deletion request 939 ...
processing deletion request 940 ...
processing deletion request 941 ...
processing deletion request 942 ...
processing deletion request 

In [43]:
st = time.time()
for d in range(len(dels)):
    print("processing deletion request {} ...".format(d+1))
    qkm.qdelete(dels[d]) 
en = time.time()

print(en-st)

processing deletion request 1 ...
no retrain
9833 12008


processing deletion request 2 ...
no retrain
9998 12007


processing deletion request 3 ...
no retrain
10013 12006


processing deletion request 4 ...
no retrain
11910 12005


processing deletion request 5 ...
no retrain
7092 12004


processing deletion request 6 ...
no retrain
304 12003


processing deletion request 7 ...
no retrain
11351 12002


processing deletion request 8 ...
no retrain
8311 12001


processing deletion request 9 ...
no retrain
6154 12000


processing deletion request 10 ...
no retrain
249 11999


processing deletion request 11 ...
no retrain
6164 11998


processing deletion request 12 ...
no retrain
9051 11997


processing deletion request 13 ...
no retrain
7724 11996


processing deletion request 14 ...
no retrain
2396 11995


processing deletion request 15 ...
no retrain
4501 11994


processing deletion request 16 ...
no retrain
935 11993


processing deletion request 17 ...
no retrain
2682 11992


proces

no retrain
5295 11755


processing deletion request 255 ...
no retrain
6367 11754


processing deletion request 256 ...
no retrain
6368 11753


processing deletion request 257 ...
no retrain
4805 11752


processing deletion request 258 ...
no retrain
5634 11751


processing deletion request 259 ...
no retrain
551 11750


processing deletion request 260 ...
no retrain
5048 11749


processing deletion request 261 ...
no retrain
6779 11748


processing deletion request 262 ...
no retrain
10217 11747


processing deletion request 263 ...
no retrain
7529 11746


processing deletion request 264 ...
no retrain
5459 11745


processing deletion request 265 ...
no retrain
4134 11744


processing deletion request 266 ...
no retrain
10654 11743


processing deletion request 267 ...
no retrain
7757 11742


processing deletion request 268 ...
no retrain
1265 11741


processing deletion request 269 ...
no retrain
11941 11740


processing deletion request 270 ...
no retrain
5334 11739


processing del

no retrain
6587 11536


processing deletion request 474 ...
no retrain
2671 11535


processing deletion request 475 ...
no retrain
226 11534


processing deletion request 476 ...
no retrain
3905 11533


processing deletion request 477 ...
no retrain
5276 11532


processing deletion request 478 ...
no retrain
2334 11531


processing deletion request 479 ...
no retrain
7129 11530


processing deletion request 480 ...
no retrain
8668 11529


processing deletion request 481 ...
no retrain
5619 11528


processing deletion request 482 ...
no retrain
6619 11527


processing deletion request 483 ...
no retrain
10047 11526


processing deletion request 484 ...
no retrain
8004 11525


processing deletion request 485 ...
no retrain
1567 11524


processing deletion request 486 ...
no retrain
9126 11523


processing deletion request 487 ...
no retrain
1581 11522


processing deletion request 488 ...
no retrain
9460 11521


processing deletion request 489 ...
no retrain
8724 11520


processing delet

processing deletion request 670 ...
no retrain
7069 11339


processing deletion request 671 ...
no retrain
9366 11338


processing deletion request 672 ...
no retrain
2080 11337


processing deletion request 673 ...
no retrain
4571 11336


processing deletion request 674 ...
no retrain
3952 11335


processing deletion request 675 ...
no retrain
2578 11334


processing deletion request 676 ...
no retrain
3810 11333


processing deletion request 677 ...
no retrain
4564 11332


processing deletion request 678 ...
no retrain
1985 11331


processing deletion request 679 ...
no retrain
2554 11330


processing deletion request 680 ...
no retrain
1555 11329


processing deletion request 681 ...
no retrain
7835 11328


processing deletion request 682 ...
no retrain
7962 11327


processing deletion request 683 ...
no retrain
1979 11326


processing deletion request 684 ...
no retrain
4483 11325


processing deletion request 685 ...
no retrain
7883 11324


processing deletion request 686 ...
no r

8184 11072


processing deletion request 938 ...
no retrain
2870 11071


processing deletion request 939 ...
no retrain
4180 11070


processing deletion request 940 ...
no retrain
1970 11069


processing deletion request 941 ...
no retrain
3327 11068


processing deletion request 942 ...
no retrain
3100 11067


processing deletion request 943 ...
no retrain
1904 11066


processing deletion request 944 ...
no retrain
4442 11065


processing deletion request 945 ...
no retrain
10006 11064


processing deletion request 946 ...
no retrain
6532 11063


processing deletion request 947 ...
no retrain
10482 11062


processing deletion request 948 ...
no retrain
5441 11061


processing deletion request 949 ...
no retrain
9117 11060


processing deletion request 950 ...
no retrain
4101 11059


processing deletion request 951 ...
no retrain
8501 11058


processing deletion request 952 ...
no retrain
9994 11057


processing deletion request 953 ...
no retrain
6534 11056


processing deletion reque

In [37]:
st = time.time()
for d in range(len(dels)):
    print("processing deletion request {} ...".format(d+1))
    km.delete(dels[d])
    print(time.time()-st)
en = time.time()

print(en-st)

processing deletion request 1 ...
2.452486038208008
processing deletion request 2 ...
4.842547416687012
processing deletion request 3 ...
7.232616662979126
processing deletion request 4 ...
9.544607877731323
processing deletion request 5 ...
12.037412881851196
processing deletion request 6 ...
14.333722829818726
processing deletion request 7 ...
16.73944067955017
processing deletion request 8 ...
19.23983407020569
processing deletion request 9 ...
21.551825284957886
processing deletion request 10 ...
23.86378574371338
processing deletion request 11 ...
26.175715923309326
processing deletion request 12 ...
28.721996545791626
processing deletion request 13 ...
31.002749919891357
processing deletion request 14 ...
33.37716054916382
processing deletion request 15 ...
35.689151763916016
processing deletion request 16 ...
37.923006772994995
processing deletion request 17 ...
40.14124369621277
processing deletion request 18 ...
42.31260681152344
processing deletion request 19 ...
44.671431541

353.42864203453064
processing deletion request 154 ...
355.8187108039856
processing deletion request 155 ...
358.1462926864624
processing deletion request 156 ...
360.23958563804626
processing deletion request 157 ...
362.37971091270447
processing deletion request 158 ...
364.58232140541077
processing deletion request 159 ...
366.6912043094635
processing deletion request 160 ...
369.03437781333923
processing deletion request 161 ...
371.3463623523712
processing deletion request 162 ...
373.48649549484253
processing deletion request 163 ...
375.8140459060669
processing deletion request 164 ...
377.9698226451874
processing deletion request 165 ...
380.1880497932434
processing deletion request 166 ...
382.53122878074646
processing deletion request 167 ...
384.89008378982544
processing deletion request 168 ...
387.12393856048584
processing deletion request 169 ...
389.2484440803528
processing deletion request 170 ...
391.3885691165924
processing deletion request 171 ...
393.59114813804626


691.1936240196228
processing deletion request 306 ...
693.5524163246155
processing deletion request 307 ...
695.9893791675568
processing deletion request 308 ...
698.2856879234314
processing deletion request 309 ...
700.4883289337158
processing deletion request 310 ...
702.7534201145172
processing deletion request 311 ...
704.9091737270355
processing deletion request 312 ...
707.0492680072784
processing deletion request 313 ...
709.2050466537476
processing deletion request 314 ...
711.3295502662659
processing deletion request 315 ...
713.4852652549744
processing deletion request 316 ...
715.6254224777222
processing deletion request 317 ...
717.7967975139618
processing deletion request 318 ...
719.9994027614594
processing deletion request 319 ...
722.1395275592804
processing deletion request 320 ...
724.2796547412872
processing deletion request 321 ...
726.4510231018066
processing deletion request 322 ...
728.6536350250244
processing deletion request 323 ...
730.8250017166138
processing

1025.612489938736
processing deletion request 458 ...
1028.002525806427
processing deletion request 459 ...
1030.1385753154755
processing deletion request 460 ...
1032.402235031128
processing deletion request 461 ...
1034.4497845172882
processing deletion request 462 ...
1036.5580534934998
processing deletion request 463 ...
1038.7103741168976
processing deletion request 464 ...
1040.7738764286041
processing deletion request 465 ...
1042.8955283164978
processing deletion request 466 ...
1045.160593032837
processing deletion request 467 ...
1047.2695081233978
processing deletion request 468 ...
1049.3627624511719
processing deletion request 469 ...
1051.643455505371
processing deletion request 470 ...
1053.8929615020752
processing deletion request 471 ...
1056.0487086772919
processing deletion request 472 ...
1058.2825317382812
processing deletion request 473 ...
1060.5320408344269
processing deletion request 474 ...
1062.7190279960632
processing deletion request 475 ...
1064.9528808593

1348.4413630962372
processing deletion request 608 ...
1350.6439442634583
processing deletion request 609 ...
1352.6888101100922
processing deletion request 610 ...
1354.6964428424835
processing deletion request 611 ...
1356.7482974529266
processing deletion request 612 ...
1358.7951710224152
processing deletion request 613 ...
1360.8542032241821
processing deletion request 614 ...
1362.9090523719788
processing deletion request 615 ...
1365.0804269313812
processing deletion request 616 ...
1367.329866886139
processing deletion request 617 ...
1369.40753698349
processing deletion request 618 ...
1371.4695551395416
processing deletion request 619 ...
1373.7814853191376
processing deletion request 620 ...
1375.8591258525848
processing deletion request 621 ...
1378.202359676361
processing deletion request 622 ...
1380.49866771698
processing deletion request 623 ...
1382.5450961589813
processing deletion request 624 ...
1384.8101620674133
processing deletion request 625 ...
1386.95031905174

1673.266226053238
processing deletion request 758 ...
1675.5000793933868
processing deletion request 759 ...
1677.5777506828308
processing deletion request 760 ...
1679.6709814071655
processing deletion request 761 ...
1681.733032464981
processing deletion request 762 ...
1683.9824755191803
processing deletion request 763 ...
1686.1070110797882
processing deletion request 764 ...
1688.3252124786377
processing deletion request 765 ...
1690.5903091430664
processing deletion request 766 ...
1692.839783191681
processing deletion request 767 ...
1695.198608636856
processing deletion request 768 ...
1697.463705778122
processing deletion request 769 ...
1699.6351051330566
processing deletion request 770 ...
1701.6971235275269
processing deletion request 771 ...
1703.8841145038605
processing deletion request 772 ...
1706.0554814338684
processing deletion request 773 ...
1708.2737176418304
processing deletion request 774 ...
1710.60129404068
processing deletion request 775 ...
1712.772668123245

1988.966510772705
processing deletion request 908 ...
1991.1690893173218
processing deletion request 909 ...
1993.4341859817505
processing deletion request 910 ...
1995.4649868011475
processing deletion request 911 ...
1997.5270116329193
processing deletion request 912 ...
1999.5577881336212
processing deletion request 913 ...
2001.619775056839
processing deletion request 914 ...
2003.6662056446075
processing deletion request 915 ...
2005.7907090187073
processing deletion request 916 ...
2007.8839716911316
processing deletion request 917 ...
2010.0396869182587
processing deletion request 918 ...
2012.1010553836823
processing deletion request 919 ...
2014.120674610138
processing deletion request 920 ...
2016.1300446987152
processing deletion request 921 ...
2018.116307258606
processing deletion request 922 ...
2020.0640215873718
processing deletion request 923 ...
2022.075365304947
processing deletion request 924 ...
2024.1061358451843
processing deletion request 925 ...
2026.1681301593